In [12]:
import os 
from dotenv import load_dotenv

from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import JsonOutputParser

load_dotenv()

True

In [3]:
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY

In [5]:
from src.agent import llm
from src.prompts import base_template, output_sample
from src.schema import NetworkConfig
from src.tools import dict_to_json

In [6]:
sample = "I want a network diagram that clearly illustrates our security posture. I need to see how our firewalls are configured and where they sit within the network, along with any other security devices like intrusion detection systems (IDS). I'd like to see the flow of traffic between our internal network and the internet, showing any DMZs or other segmentation measures. The diagram should also highlight any potential vulnerabilities, like open ports or exposed services, so we can prioritize remediation efforts."

In [7]:
loader = CSVLoader(file_path="./vm_info.csv")
docs = loader.load()
docs

[Document(page_content='Unique_ID: 1\nPlatform: Windows\nDistributor: Windows Server\nDescription: A general-purpose VM suitable for Windows-based workloads\nMin_CPU: 2\nMin_RAM_GB: 4\nMin_Storage_GB: 50\nInstalled_Software: OS:Windows Server 2019, IIS:10.0, SQL Server:2019\nImage_Version: 2019', metadata={'source': './vm_info.csv', 'row': 0}),
 Document(page_content='Unique_ID: 2\nPlatform: Linux\nDistributor: Ubuntu\nDescription: A lightweight VM for Linux development environments\nMin_CPU: 1\nMin_RAM_GB: 2\nMin_Storage_GB: 20\nInstalled_Software: OS:Ubuntu 20.04, Apache:2.4, MySQL:8.0\nImage_Version: 20.04', metadata={'source': './vm_info.csv', 'row': 1}),
 Document(page_content='Unique_ID: 3\nPlatform: Windows\nDistributor: Windows Pro\nDescription: High-performance VM for Windows desktop applications\nMin_CPU: 4\nMin_RAM_GB: 8\nMin_Storage_GB: 100\nInstalled_Software: OS:Windows 10 Pro, Office:2019, Visual Studio:2019\nImage_Version: win10_pro', metadata={'source': './vm_info.csv'

In [37]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10, chunk_overlap=8)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=GoogleGenerativeAIEmbeddings(model='models/embedding-001'))
vectorstore

In [38]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [39]:
retriever.invoke(sample)

[Document(page_content='Description: Firewall server for network security and traffic monitoring\nMin_CPU: 2\nMin_RAM_GB: 4', metadata={'row': 12, 'source': './vm_info.csv'}),
 Document(page_content='Distributor: Ubuntu\nDescription: Firewall server for network security and traffic monitor3ing', metadata={'row': 13, 'source': './vm_info.csv'}),
 Document(page_content='Unique_ID: 13\nPlatform: Windows\nDistributor: Windows Server\nDescription: Firewall server for network security and traffic monitoring\nMin_CPU: 2\nMin_RAM_GB: 4\nMin_Storage_GB: 50\nInstalled_Software: OS:Windows Server 2019, Windows Defender:10.0, Firewall Software:2.0\nImage_Version: 2019', metadata={'row': 12, 'source': './vm_info.csv'}),
 Document(page_content='Unique_ID: 13\nPlatform: Windows\nDistributor: Windows Server\nDescription: Firewall server for network security and traffic monitoring\nMin_CPU: 2\nMin_RAM_GB: 4\nMin_Storage_GB: 50\nInstalled_Software: OS:Windows Server 2019, Windows Defender:10.0, Firewall

In [40]:
json_parser = JsonOutputParser(pydantic_object=NetworkConfig)

In [41]:
template = """
Your task is to design and implement a workflow that generates a network diagram featuring
virtual machines (VMs) based on customer specifications. The workflow should use a database
of VM types, detailing their descriptions, minimum specifications, and installed software, and
produce a JSON output that represents the network diagram, including VLANs and the specified
machines.


Sample Responses:
{json_data}


Question: {question} 

Context: {context} 

Do not answer out of the Sample Response schema.

Answer:

"""

rag_prompt = PromptTemplate.from_template(template, 
                                            partial_variables={"format_instructions": json_parser.get_format_instructions()})

In [42]:
rag_chain = (
    {"context": retriever, "json_data": lambda x: output_sample, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | json_parser
)

In [43]:
rag_chain.invoke(sample)

{'action': 'create',
 'env_id': '9876abdc5432',
 'network_portgroup': {'pg_internet': {'allow_promiscuous': False,
   'default_gateway': '172.16.0.1',
   'dhcp_range': [],
   'netmask': '24',
   'pg_name': 'pg_internet',
   'subnet': '172.16.0.0',
   'vlan_id': 1},
  'pg_dmz': {'allow_promiscuous': False,
   'default_gateway': '172.17.0.1',
   'dhcp_range': [],
   'netmask': '24',
   'pg_name': 'pg_dmz',
   'subnet': '172.17.0.0',
   'vlan_id': 2},
  'pg_internal': {'allow_promiscuous': False,
   'default_gateway': '172.18.0.1',
   'dhcp_range': ['172.18.0.100', '172.18.0.200'],
   'netmask': '24',
   'pg_name': 'pg_internal',
   'subnet': '172.18.0.0',
   'vlan_id': 3}},
 'resource_id': 'abcdef123456',
 'vm_instance': {'firewall1': {'cpu_number': '2',
   'disk_size_gb': '50',
   'gateway': '172.16.0.1',
   'hostname': 'firewall1',
   'image': 'Windows Server 2019',
   'instance': 1,
   'network': {'eth0': {'ipv4_address': ['172.16.0.10'],
     'ipv4_netmask': ['24'],
     'portgroup':